In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext cudf.pandas
import pandas as pd

import numpy as np
import gc

gc.collect()

0

In [ ]:
gc.collect()

190

In [ ]:
import os

folder_path = '/content/drive/MyDrive/credit/train/1.회원정보'

parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

df_list = [pd.read_parquet(file) for file in parquet_files]
df = pd.concat(df_list, ignore_index=True)

pd.set_option('display.max_columns', None)

print(df.head())
gc.collect()

     기준년월            ID  남녀구분코드   연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
0  201808  TRAIN_000000       2  40대       D          1             1   
1  201808  TRAIN_000001       1  30대       E          1             1   
2  201808  TRAIN_000002       1  30대       C          1             1   
3  201808  TRAIN_000003       2  40대       D          1             1   
4  201808  TRAIN_000004       2  40대       E          1             1   

   회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  소지카드수_이용가능_신용   입회일자_신용  입회경과개월수_신용  \
0              0        1            1              1  20130101          68   
1              1        1            1              1  20170801          13   
2              0        1            1              1  20080401         125   
3              0        1            2              2  20160501          28   
4              1        1            1              1  20180601           3   

   회원여부_연체  이용거절여부_카드론  동의여부_한도증액안내  수신거부여부_TM  수신거부여부_DM  수신거부여부_메일  \
0        0    

1270

In [ ]:
import os

folder_path_2 = '/content/drive/MyDrive/credit/train/2.신용정보'

parquet_files_2 = [os.path.join(folder_path_2, f) for f in os.listdir(folder_path_2) if f.endswith('.parquet')]

df_list_2 = [pd.read_parquet(file) for file in parquet_files_2]
df_2 = pd.concat(df_list_2, ignore_index=True)

pd.set_option('display.max_columns', None)

print(df_2.head())
gc.collect()

     기준년월            ID  최초한도금액  카드이용한도금액  CA한도금액  일시상환론한도금액  월상환론한도금액  \
0  201808  TRAIN_000000       0     19796    6293          0         0   
1  201808  TRAIN_000001       0      9997    5222      41681     91029   
2  201808  TRAIN_000002       0     88174   29594          0         0   
3  201808  TRAIN_000003       0     19679    7990          0         0   
4  201808  TRAIN_000004       0    185750   52815      47997    155900   

   CA이자율_할인전  CL이자율_할인전  RV일시불이자율_할인전  RV현금서비스이자율_할인전      RV신청일자  RV약정청구율  \
0  22.995351  18.305938     17.600156       21.237342        <NA>   0.0000   
1  14.797066  14.842079     10.597465       13.637134  20171114.0  99.9998   
2  21.900136  18.045304     16.960661       21.224218  20120328.0  99.9998   
3  22.995698  22.999954     19.371048       22.999954        <NA>   0.0000   
4  14.715828  10.897486     10.530790       10.578001        <NA>   0.0000   

   RV최소결제비율 자발한도감액횟수_R12M  자발한도감액금액_R12M  자발한도감액후경과월  강제한도감액횟수_R12M  \
0  19.99996    

0

In [ ]:
numeric_df = df.select_dtypes(include=['int64', 'float64'])
numeric_df_2 = df_2.select_dtypes(include=['int64', 'float64'])

In [ ]:
def load_parquet_folder(folder_path):
    parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]
    df_list = [pd.read_parquet(file) for file in parquet_files]
    df = pd.concat(df_list, ignore_index=True)

    pd.set_option('display.max_columns', None)
    print(df.head())
    gc.collect()

    return df

In [ ]:
def merge_df(df_source, df_target):
    # 숫자형(numeric)이 아닌 컬럼들만 고르기
    non_numeric_cols = df_source.select_dtypes(exclude=['number'])
    # 두 데이터프레임 합치기 (column 기준, index 맞춰서)
    merged_df = pd.concat([df_target.reset_index(drop=True), non_numeric_cols.reset_index(drop=True)], axis=1)
    return merged_df

In [ ]:
def merge_high_corr_columns(df, threshold=0.7):
    """
    반복적으로 상관계수가 높은 열을 통합하여 새로운 데이터프레임 생성
    :param df: 입력 데이터프레임
    :param threshold: 상관계수 기준값 (기본값: 0.7)
    :return: 통합된 데이터프레임
    """
    while True:
        # 상관행렬 계산
        corr_matrix = df.corr()

        # 상관계수 절대값 기준으로 높은 열 쌍 찾기
        high_corr_pairs = [
            (col1, col2) for col1 in corr_matrix.columns
            for col2 in corr_matrix.columns
            if col1 != col2 and abs(corr_matrix.loc[col1, col2]) >= threshold
        ]

        # 높은 상관계수 쌍이 없으면 종료
        if not high_corr_pairs:
            break

        # 첫 번째 높은 상관계수 쌍 선택 (임의로 선택)
        col1, col2 = high_corr_pairs[0]

        # 새로운 열 생성 (평균 또는 합)
        new_col_name = f"{col1}+{col2}_new"
        df[new_col_name] = df[[col1, col2]].mean(axis=1)

        # 기존 열 제거
        df = df.drop(columns=[col1, col2])

    return df

In [ ]:
df_3 = load_parquet_folder('/content/drive/MyDrive/credit/train/3.승인매출정보')
df_4 = load_parquet_folder('/content/drive/MyDrive/credit/train/4.청구입금정보')

numeric_df_3 = df.select_dtypes(include=['int64', 'float64'])
numeric_df_4 = df_2.select_dtypes(include=['int64', 'float64'])

     기준년월            ID  최종이용일자_기본  최종이용일자_신판  최종이용일자_CA  최종이용일자_카드론  \
0  201807  TRAIN_000000   20180719   20180713   20180719       10101   
1  201807  TRAIN_000001   20180719   20180719   20170728    20170327   
2  201807  TRAIN_000002   20180706   20180706   20180706    20151119   
3  201807  TRAIN_000003   20180721   20180715   20180721       10101   
4  201807  TRAIN_000004   20180124   20180124      10101       10101   

   최종이용일자_체크  최종이용일자_일시불  최종이용일자_할부  이용건수_신용_B0M  이용건수_신판_B0M  이용건수_일시불_B0M  \
0   20180203    20180709   20180713           11            9             8   
1      10101    20180719   20171231           13           13            13   
2   20141230    20180706   20180627           12           10            10   
3   20141111    20180704   20180715            6            4             3   
4   20180512    20180124      10101           -2           -2            -2   

   이용건수_할부_B0M  이용건수_할부_유이자_B0M  이용건수_할부_무이자_B0M  이용건수_부분무이자_B0M  이용건수_CA_B0M  \
0          

In [ ]:
merged_df_3 = merge_high_corr_columns(numeric_df_3)
merged_df_4 = merge_high_corr_columns(numeric_df_4)

In [ ]:
%%time
# 함수 실행
merged_df = merge_high_corr_columns(numeric_df)
merged_df_2 = merge_high_corr_columns(numeric_df_2)
# 결과 확인
print(merged_df)
print(merged_df_2)

           기준년월  남녀구분코드  회원여부_이용가능  회원여부_이용가능_CA  회원여부_이용가능_카드론  소지여부_신용  \
0        201808       2          1             1              0        1   
1        201808       1          1             1              1        1   
2        201808       1          1             1              0        1   
3        201808       2          1             1              0        1   
4        201808       2          1             1              1        1   
...         ...     ...        ...           ...            ...      ...   
2399995  201812       2          1             1              1        1   
2399996  201812       2          1             1              1        1   
2399997  201812       1          1             1              0        1   
2399998  201812       1          1             1              1        1   
2399999  201812       2          1             1              0        1   

         회원여부_연체  이용거절여부_카드론  동의여부_한도증액안내  수신거부여부_SMS  마케팅동의여부  유효카드수_체크_가족  \
0       

In [ ]:
merged_df.to_parquet('/content/drive/MyDrive/credit/preprocess/corr/1_회원정보.parquet')
merged_df_2.to_parquet('/content/drive/MyDrive/credit/preprocess/corr/2_신용정보.parquet')

In [ ]:
merged_df = pd.read_parquet('/content/drive/MyDrive/credit/preprocess/corr/1_회원정보.parquet')
merged_df_2 = pd.read_parquet('/content/drive/MyDrive/credit/preprocess/corr/2_신용정보.parquet')

In [ ]:
#merged_df = merge_df(df, merged_df)
#merged_df_2 = merge_df(df_2, merged_df_2)
merged_df_3 = merge_df(df_3, merged_df_3)
merged_df_4 = merge_df(df_4, merged_df_4)

In [ ]:
merged_df_3.to_parquet('/content/drive/MyDrive/credit/preprocess/corr/3_승인매출정보.parquet')
merged_df_4.to_parquet('/content/drive/MyDrive/credit/preprocess/corr/4_청구입금정보.parquet')

In [ ]:
merged_df_2

,기준년월,최초한도금액,일시상환론한도금액,월상환론한도금액,RV신청일자,자발한도감액금액_R12M,자발한도감액후경과월,강제한도감액금액_R12M,월상환론상향가능한도금액,일시불ONLY전환가능여부,특별한도보유여부_R3M,연체감액여부_R3M,한도심사요청후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,RV약정청구율_new,강제한도감액횟수_R12M_new,한도증액금액_R12M_new,상향가능한도금액_new,한도요청거절건수_new,카드이용한도금액_new_new,CA이자율_할인전_new_new,ID,자발한도감액횟수_R12M,한도증액횟수_R12M,카드론동의여부,RV전환가능여부,한도심사요청건수
0,201808,0,0,0,NaN,0,12,0,0,0,0,0,3,0,0,0,0,5.000000e+07,6.0,6.0,0.0,1.5,16938.00,20.180000,TRAIN_000000,0회,0회,Y,N,0회
1,201808,0,41681,91029,20171114.0,0,12,0,0,1,0,0,3,0,0,0,0,2.109999e+02,6.0,6.0,2.0,1.5,8805.00,12.939106,TRAIN_000001,0회,0회,Y,Z,0회
2,201808,0,0,0,20120328.0,0,12,2,0,0,0,0,3,0,0,0,0,1.229500e+03,5.0,6.0,0.0,1.5,68308.75,19.759460,TRAIN_000002,0회,0회,Y,N,0회
3,201808,0,0,0,NaN,0,12,0,0,0,0,0,3,0,0,0,0,5.000000e+07,6.0,6.0,0.0,1.5,17350.50,21.795278,TRAIN_000003,0회,0회,Y,N,0회
4,201808,0,47997,155900,NaN,0,12,0,0,1,0,0,3,0,0,0,0,5.000000e+07,6.0,6.0,0.0,1.5,143064.00,11.541922,TRAIN_000004,0회,0회,Y,Z,0회
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,0,0,54445,NaN,0,12,0,0,1,0,0,3,0,0,0,0,5.000000e+07,6.0,6.0,2.0,1.5,18121.50,12.306028,TRAIN_399995,0회,0회,Y,Z,0회
2399996,201812,0,0,156800,NaN,0,12,0,0,1,0,0,3,0,0,0,0,5.000000e+07,6.0,4.5,6.5,1.5,68128.25,13.218042,TRAIN_399996,0회,1회이상,Y,Z,0회
2399997,201812,0,0,0,NaN,0,12,0,0,0,0,0,3,0,0,0,0,5.000000e+07,6.0,6.0,0.0,1.5,49182.50,17.438734,TRAIN_399997,0회,0회,Y,N,0회
2399998,201812,0,90004,180906,NaN,0,12,0,0,1,0,0,3,0,0,0,0,5.000000e+07,6.5,6.0,2.5,1.5,8558.25,12.254935,TRAIN_399998,0회,0회,Y,Z,0회
